In [1]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from operator import itemgetter

In [37]:
# Python with requests and BeautifulSoup libraries is the best choice to parse an HTML page in the web.
url = "https://imsdb.com/scripts/Godfather.html"
response = requests.get(url)
html_content = response.content

# Create a BeautifulSoup object to parse an HTML
soup = BeautifulSoup(markup = html_content, features = "html.parser")

# Find all b's in the content
all_bs = soup.body.find_all('table')[1].tr.pre.pre.find_all('b')

In [38]:
all_bs

[<b>	THE GODFATHER
 </b>,
 <b>	_____________
 </b>,
 <b>	MARIO PUZO
 </b>,
 <b>	FRANCIS FORD COPPOLA
 </b>,
 <b>THIRD DRAFT				PARAMOUNT PICTURES
 </b>,
 <b>	INT DAY: DON'S OFFICE (SUMMER 1945)
 </b>,
 <b>				  THE GODFATHER
 </b>,
 <b>				BONASERA
 </b>,
 <b>				BONASERA
 </b>,
 <b>				BONASERA
 </b>,
 <b>				DON CORLEONE
 </b>,
 <b>				BONASERA
 </b>,
 <b>				DON CORLEONE
 </b>,
 <b>				DON CORLEONE
 </b>,
 <b>				BONASERA
 </b>,
 <b>				DON CORLEONE
 </b>,
 <b>				BONASERA
 </b>,
 <b>				DON CORLEONE
 </b>,
 <b>				BONASERA
 </b>,
 <b>				DON CORLEONE
 </b>,
 <b>				BONASERA
 </b>,
 <b>				DON CORLEONE
 </b>,
 <b>				BONASERA
 </b>,
 <b>				DON CORLEONE
 </b>,
 <b>				BONASERA
 </b>,
 <b>				DON CORLEONE
 </b>,
 <b>	EXT DAY: MALL (SUMMER 1945)
 </b>,
 <b>				DON CORLEONE
 </b>,
 <b>				SONNY
 </b>,
 <b>				DON CORLEONE
 </b>,
 <b>				SONNY
 </b>,
 <b>				SANDRA
 </b>,
 <b>	EXT DAY: MALL ENTRANCE (SUMMER 1945)
 </b>,
 <b>				CLEMENZA
 </b>,
 <b>				PAULIE
 </b>,
 <b>				CLEMENZA
 </b

In [39]:
raw_character_text_list, spoken_words_list = [], []

for idx, s in enumerate(all_bs):
    raw_character_text_list.append(s.text)
    spoken_words_list.append(s.next_sibling)   

In [40]:
gf_df = pd.DataFrame({'raw_character_text' : raw_character_text_list, 'spoken_words' : spoken_words_list})

gf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1191 entries, 0 to 1190
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   raw_character_text  1191 non-null   object
 1   spoken_words        1191 non-null   object
dtypes: object(2)
memory usage: 18.7+ KB


In [42]:
gf_df.head()

,raw_character_text,spoken_words
0,\tTHE GODFATHER\r\n,[\t_____________\r\n]
1,\t_____________\r\n,\r\n\tScreenplay\r\n\r\n\tby\r\n\r\n
2,\tMARIO PUZO\r\n,\r\n\tand\r\n\r\n
3,\tFRANCIS FORD COPPOLA\r\n,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r...
4,THIRD DRAFT\t\t\t\tPARAMOUNT PICTURES\r\n,\t\t\t\t\t1 Gulf and Western Plaza\r\nMarch 29...
